# **Adding 3 Vectors Performance Task Solution by Wael KABOUK, ID:Y235050005**

##Preparing CUDA Enviorement

In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin
%load_ext nvcc_plugin

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-bmz0g2jo
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-bmz0g2jo
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4294 sha256=ab16e73e0c37efc6d768f6b97b71f5372238ed6910191c55dcc1f784a7507cf7
  Stored in directory: /tmp/pip-ephem-wheel-cache-brhyc9_d/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out
The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


## Implementing The Code And Writing it To A File



In [ ]:
%%writefile addVectorsTask.cu
#include "device_launch_parameters.h"
#include <stdlib.h>
#include <stdio.h>
#include <cuda_runtime.h>
#include <cstdlib>
#include <chrono>
#include <time.h>

#define CHECK(call)                                            \
{                                                               \
    const cudaError_t error = call;                             \
    if (error != cudaSuccess)                                   \
    {                                                           \
        fprintf(stderr, "Error: %s:%d, ", __FILE__, __LINE__); \
        fprintf(stderr, "code: %d, reason: %s\n", error,        \
        cudaGetErrorString(error));                     \
        exit(EXIT_FAILURE);                                             \
    }                                                           \
}

// Kernel function for vector addition on the GPU
__global__ void vectorAdd(const int* A, const int* B, const int* C, int* D, int numElements)
{
    int i = blockDim.x * blockIdx.x + threadIdx.x;

    if (i < numElements)
    {
        D[i] = A[i] + B[i] + C[i];
    }
}

// Function to generate a random integer in a given range
int randInt(int from, int to)
{
    return rand() % (to - from + 1) + from;
}

// Function to print the elements of a vector
void printVector(int* vector, int length)
{
    for (int i = 0; i < length; i++) {
        printf("%d ", vector[i]);
    }
    printf("\n");
}

// Function for vector addition on the CPU
void cpuVectorAdd(const int* A, const int* B, const int* C, int* D, int numElements)
{
    for (int i = 0; i < numElements; i++) {
        D[i] = A[i] + B[i] + C[i];
    }
}

int main(int argc, char** argv)
{
    // Initialize random seed
    srand((unsigned)time(NULL));

    // Error code to check return values for CUDA calls
    cudaError_t err = cudaSuccess;

    // Vector length and size calculation
    int numElements = pow(atoi(argv[1]), atoi(argv[2]));
    size_t size = numElements * sizeof(int);

    printf("Vector addition of %d elements\n", numElements);

    // Allocate host input and output vectors
    int* h_A = (int*)malloc(size);
    int* h_B = (int*)malloc(size);
    int* h_C = (int*)malloc(size);
    int* h_D = (int*)malloc(size);

    // Verify host vector allocations
    if (h_A == NULL || h_B == NULL || h_C == NULL || h_D == NULL)
    {
        fprintf(stderr, "Failed to allocate host vectors!\n");
        exit(EXIT_FAILURE);
    }

    // Initialize host input vectors
    for (int i = 0; i < numElements; i++)
    {
        h_A[i] = randInt(1, 100);
        h_B[i] = randInt(1, 100);
        h_C[i] = randInt(1, 100);
    }

    // Allocate device input and output vectors
    int* d_A = NULL;
    int* d_B = NULL;
    int* d_C = NULL;
    int* d_D = NULL;

    err = cudaMalloc((void**)&d_A, size);
    CHECK(err);

    err = cudaMalloc((void**)&d_B, size);
    CHECK(err);

    err = cudaMalloc((void**)&d_C, size);
    CHECK(err);

    err = cudaMalloc((void**)&d_D, size);
    CHECK(err);

    // Copy host input vectors to device
    printf("Copy input data from host memory to CUDA device\n");
    err = cudaMemcpy(d_A, h_A, size, cudaMemcpyHostToDevice);
    CHECK(err);

    err = cudaMemcpy(d_B, h_B, size, cudaMemcpyHostToDevice);
    CHECK(err);

    err = cudaMemcpy(d_C, h_C, size, cudaMemcpyHostToDevice);
    CHECK(err);

    // CUDA event setup for timing
    cudaEvent_t start, stop;
    float elapsedTime;

    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    cudaEventRecord(start, 0);

    // Launch the Vector Add CUDA Kernel
    int threadsPerBlock = 512;
    int blocksPerGrid = (numElements + threadsPerBlock - 1) / threadsPerBlock;

    printf("CUDA kernel launch with %d blocks of %d threads\n", blocksPerGrid, threadsPerBlock);
    vectorAdd << < blocksPerGrid, threadsPerBlock >> > (d_A, d_B, d_C, d_D, numElements);
    err = cudaGetLastError();
    CHECK(err);

    // Record CUDA event stop time
    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);
    // Calculate and print GPU elapsed time
    cudaEventElapsedTime(&elapsedTime, start, stop);
    printf("Time on device: %f ms\n", elapsedTime);



    clock_t cpuStart, cpuStop;
    float cpuDuration;

    // Record the start time for CPU
    cpuStart = clock();

    // Perform vector addition on CPU
    cpuVectorAdd(h_A, h_B, h_C, h_D, numElements);

    // Record the stop time for CPU
    cpuStop = clock();

    // Calculate elapsed time on the CPU
    cpuDuration = ((float)(cpuStop - cpuStart)) * 1000.0 / CLOCKS_PER_SEC;
    printf("Time on CPU: %f ms\n", cpuDuration);

    // Calculate and print speedup
    float speedup = cpuDuration / elapsedTime;
    printf("Speed-up: %.2f\n", speedup);

    // Copy device result vector to host
    printf("Copy output data from CUDA device to host memory\n");
    err = cudaMemcpy(h_D, d_D, size, cudaMemcpyDeviceToHost);
    CHECK(err);

    // Verify the result vector
    for (int i = 0; i < numElements; ++i)
    {
        if (fabs(h_A[i] + h_B[i] + h_C[i] - h_D[i]) > 1e-5)
        {
            fprintf(stderr, "Result verification failed at element %d!\n", i);
            exit(EXIT_FAILURE);
        }
    }

    printf("Test PASSED\n");


    // Free device global memory
    err = cudaFree(d_A);
    CHECK(err);

    err = cudaFree(d_B);
    CHECK(err);

    err = cudaFree(d_C);
    CHECK(err);

    err = cudaFree(d_D);
    CHECK(err);

    // Free host memory
    free(h_A);
    free(h_B);
    free(h_C);
    free(h_D);

    printf("Done\n");
    return 0;
}


Overwriting addVectorsTask.cu


## Compiling The code

In [ ]:
!nvcc -o addVectorsTask addVectorsTask.cu

## Executing the code with an input value of N = 2^10

In [ ]:
!./addVectorsTask 2 10

Vector addition of 1024 elements
Copy input data from host memory to CUDA device
CUDA kernel launch with 2 blocks of 512 threads
Time on device: 0.028128 ms
Time on CPU: 0.007000 ms
Speed-up: 0.25
Copy output data from CUDA device to host memory
Test PASSED
Done


## Executing the code with an input value of N = 2^12


In [ ]:
!./addVectorsTask 2 12

Vector addition of 4096 elements
Copy input data from host memory to CUDA device
CUDA kernel launch with 8 blocks of 512 threads
Time on device: 0.028672 ms
Time on CPU: 0.029000 ms
Speed-up: 1.01
Copy output data from CUDA device to host memory
Test PASSED
Done


## Executing the code with an input value of N = 2^14

In [ ]:
!./addVectorsTask 2 14

Vector addition of 16384 elements
Copy input data from host memory to CUDA device
CUDA kernel launch with 32 blocks of 512 threads
Time on device: 0.032608 ms
Time on CPU: 0.092000 ms
Speed-up: 2.82
Copy output data from CUDA device to host memory
Test PASSED
Done


## Conclusion
As vector sizes expand, the benefits of GPU acceleration become increasingly conspicuous. The experiment highlights CUDA's scalability, with the GPU's parallel architecture demonstrating exceptional prowess in managing larger datasets. This positions it as a compelling option for tasks requiring substantial computational throughput.

It is crucial to acknowledge that the observed speed-up is subject to the particular characteristics of the algorithm and the hardware configuration. Despite this, the outcomes of the experiment underscore a prevailing trend: GPU acceleration has the potential to yield significant enhancements in performance for workloads amenable to parallelization.